In [1]:
import os
import torch

In [2]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '0'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    os.environ["CUDA_VISIBLE_DEVICES"] = '0, 1, 2, 3, 4, 5, 6, 7'

In [3]:
model_checkpoint = "facebook/wav2vec2-base"
# model_checkpoint = "facebook/wav2vec2-large-xlsr-53"
batch_size = 16

In [4]:
from datasets import load_dataset, load_metric, Audio,load_from_disk

dataset = load_from_disk("./data/datasets/train")

common_voice_train = load_from_disk("./data/datasets/real_test")
common_voice_test = load_from_disk("./data/datasets/real_test")

/home/haram/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [6]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["transcription"] = re.sub(chars_to_ignore_regex, '', batch["transcription"]).lower() + " "
    return batch

In [7]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

show_random_elements(common_voice_train.remove_columns(["path","audio"]))

,transcription
0,and so he went softly up to the pig sty and reached over and grabbed the little pig by the ears the pig squealed of course but the farmer was making so much noise himself that he did not hear it
1,that poor sailor will think i've stolen his ship if i don't get home soon i wonder if those hinges are loose but the door was very strong and firmly locked
2,and appreciated only their acquired accomplishments as they appreciated the skill of their cook or the taste of their waiting woman
3,but went to sleep again somewhere else so i came back full of sadness to my father's kingdom
4,i have said enough
5,trying to sniff the cooking smells that came from the palace kitchen she told the pig to bring the doctor to the window because she wanted to speak to him so gub gub went and woke the doctor
6,and he talked to me in such a jolly way that i sat perfectly still and allowed him to measure my ears and my legs so that he could cut the fur into the proper form
7,and blundered into the palace garden by mistake but couldn't you guide them asked chee chee and he began to scold the parrot for letting them get lost while he was away looking for the cocoanuts
8,and all the animals on the ship began to laugh and dance about in the rushing air for when they looked back at the pirates ship they could see that it was growing smaller now instead of bigger
9,i have a feeling it isn't a friendly ship i am afraid there is more trouble coming to us jip who was lying near taking a nap in the sun began to growl and talk in his sleep


In [8]:
def extract_all_chars(batch):
  all_text = " ".join(batch["transcription"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [9]:
vocab_train = common_voice_train.map(
  extract_all_chars, batched=True, 
  batch_size=-1, keep_in_memory=True, 
  remove_columns=common_voice_train.column_names
)
vocab_test = common_voice_test.map(
  extract_all_chars, batched=True, 
  batch_size=-1, keep_in_memory=True, 
  remove_columns=common_voice_test.column_names
)

Map: 100%|██████████| 200/200 [00:00<00:00, 36237.45 examples/s]


In [10]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [11]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'p': 0,
 'w': 1,
 'q': 2,
 'e': 3,
 'n': 4,
 'j': 5,
 'o': 6,
 's': 7,
 'b': 8,
 ' ': 9,
 'h': 10,
 'v': 11,
 'k': 12,
 'z': 13,
 "'": 14,
 'i': 15,
 't': 16,
 'r': 17,
 'x': 18,
 'a': 19,
 'd': 20,
 'u': 21,
 'y': 22,
 'l': 23,
 'g': 24,
 'f': 25,
 'c': 26,
 'm': 27}

In [12]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [13]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

30

In [14]:
import json
with open('./data/vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [15]:
from transformers import Wav2Vec2CTCTokenizer

# tokenizer 객체 생성
origin_tokenizer = Wav2Vec2CTCTokenizer(vocab_file="./data/vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

# tokenizer 저장!
origin_tokenizer.save_pretrained("./data/tokenizer2")

('./data/tokenizer2/tokenizer_config.json',
 './data/tokenizer2/special_tokens_map.json',
 './data/tokenizer2/vocab.json',
 './data/tokenizer2/added_tokens.json')

In [16]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained(model_checkpoint)

tokenizer_type = config.model_type if config.tokenizer_class is None else None
config = config if config.tokenizer_class is not None else None

/home/haram/.local/lib/python3.10/site-packages/transformers/configuration_utils.py:363: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [17]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
  "./data/tokenizer2",
  config=config,
  tokenizer_type=tokenizer_type,
  unk_token="[UNK]",
  pad_token="[PAD]",
  word_delimiter_token="|",
)

`use_fast` is set to `True` but the tokenizer class does not have a fast version.  Falling back to the slow version.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [18]:
model_checkpoint_name = model_checkpoint.split("/")[-1]
repo_name = "./data/test_repo"

In [19]:
# tokenizer.push_to_hub(repo_name)

In [20]:
common_voice_train[0]["path"]

'./data/test_data/100/121674/100-121674-0037.flac'

In [21]:
common_voice_train[0]["audio"]

{'path': '100-121674-0037.flac',
 'array': array([-3.05175781e-04, -1.00708008e-03, -1.83105469e-04, ...,
        -1.22070312e-04, -9.15527344e-05,  9.15527344e-05]),
 'sampling_rate': 16000}

In [22]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [23]:
common_voice_train[0]["audio"]

{'path': '100-121674-0037.flac',
 'array': array([-3.05175781e-04, -1.00708008e-03, -1.83105469e-04, ...,
        -1.22070312e-04, -9.15527344e-05,  9.15527344e-05]),
 'sampling_rate': 16000}

In [24]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

print(common_voice_train[rand_int]["transcription"])
ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

i must be a white prince you know it is very hard to change the color of a prince said the doctor one of the hardest things a magician can do 


In [25]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["transcription"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])

Target text: i must put a squeak in it said santa so he took a box of squeaks from a shelf and put one into the rabbit before he sewed it up when it was all finished he pressed the toy rabbit with his thumb and it squeaked so naturally that i jumped off the table 
Input array shape: (242000,)
Sampling rate: 16000


In [26]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)

/home/haram/.local/lib/python3.10/site-packages/transformers/configuration_utils.py:363: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [27]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [28]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["transcription"]).input_ids
    return batch

In [29]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]2024-04-22 17:58:09.493709: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-22 17:58:09.549678: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-22 17:58:09.549726: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-22 17:58:09.549778: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-22 17:58:

In [30]:
max_input_length_in_sec = 5.0
common_voice_train = common_voice_train.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

Filter: 100%|██████████| 200/200 [00:00<00:00, 46448.55 examples/s]


In [31]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [32]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [33]:
wer_metric = load_metric("wer")

/tmp/ipykernel_3881535/24688115.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")
/home/haram/.local/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for wer contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/wer/wer.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [34]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [35]:
from transformers import AutoModelForCTC
import torch.nn as nn

model = AutoModelForCTC.from_pretrained(
    model_checkpoint,
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
).to(device)

if torch.cuda.is_available():
    model = nn.DataParallel(model)
    

/home/haram/.local/lib/python3.10/site-packages/transformers/configuration_utils.py:363: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.bias', 'lm_head.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
if hasattr(model.module, "freeze_feature_extractor"):
  model.module.freeze_feature_extractor()

if hasattr(model.module, "gradient_checkpointing_enable"):
    model.module.gradient_checkpointing_enable()

/home/haram/.local/lib/python3.10/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1920: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [37]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=batch_size,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  # push_to_hub=True,
)

# training_args = TrainingArguments(
#   output_dir="./model/test",
#   per_device_train_batch_size=16,
#   gradient_accumulation_steps=2,
#   num_train_epochs=10,
#   fp16=True,
#   save_steps=10,
#   logging_steps=10,
#   learning_rate=1e-4,
#     remove_unused_columns=False
# )

In [38]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

/home/haram/.local/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

/home/haram/.local/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/haram/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
